# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [28]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,61.32,80,0,5.95,clear sky
2,2,Dikson,RU,73.5069,80.5464,-18.62,97,95,12.73,overcast clouds
3,3,Storforshei,NO,66.4043,14.5233,-6.02,77,98,6.91,overcast clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,61.77,94,80,18.99,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [29]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [30]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,75.88,92,100,17.94,overcast clouds
10,10,Urubicha,BO,-15.3833,-62.9500,72.75,97,94,0.94,overcast clouds
12,12,Harper,LR,4.3750,-7.7169,80.19,84,89,6.04,overcast clouds
15,15,Namibe,AO,-15.1961,12.1522,72.54,71,99,4.83,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [31]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,75.88,92,100,17.94,overcast clouds
10,10,Urubicha,BO,-15.3833,-62.9500,72.75,97,94,0.94,overcast clouds
12,12,Harper,LR,4.3750,-7.7169,80.19,84,89,6.04,overcast clouds
15,15,Namibe,AO,-15.1961,12.1522,72.54,71,99,4.83,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [32]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities.copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [33]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds,
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,75.88,92,100,17.94,overcast clouds,
10,10,Urubicha,BO,-15.3833,-62.9500,72.75,97,94,0.94,overcast clouds,
12,12,Harper,LR,4.3750,-7.7169,80.19,84,89,6.04,overcast clouds,
15,15,Namibe,AO,-15.1961,12.1522,72.54,71,99,4.83,overcast clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [34]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [35]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
Maiduguri - nearest hotel: Nanne and Boi Suites
Arraial Do Cabo - nearest hotel: No hotel found
Urubicha - nearest hotel: No hotel found
Harper - nearest hotel: No hotel found
Namibe - nearest hotel: Hotel Chik Chik Namibe
Victoria - nearest hotel: 四季酒店 Four Seasons Hotel
Xai-Xai - nearest hotel: No hotel found
Mahebourg - nearest hotel: Grand Bel Air
Caninde - nearest hotel: Hotel Jardineira
Saint-Philippe - nearest hotel: Le Baril
Bandarbeyla - nearest hotel: No hotel found
Tiruchchendur - nearest hotel: No hotel found
Pizarro - nearest hotel: No hotel found
Eyl - nearest hotel: فندق ايل دابي
Porto Novo - nearest hotel: Hôtel 6500 CFA
Ambilobe - nearest hotel: Hôtel National
Morros - nearest hotel: No hotel found
Monrovia - nearest hotel: Bella Casa Hotel
Dingle - nearest hotel: No hotel found
Angoche - nearest hotel: Hotel Quirimbas
Youkounkoun - nearest hotel: No hotel found
Oranjestad - nearest hotel: Hotel California
Kudahuvadhoo - nearest hotel: No hotel fo

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds,Nanne and Boi Suites
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,75.88,92,100,17.94,overcast clouds,No hotel found
10,10,Urubicha,BO,-15.3833,-62.9500,72.75,97,94,0.94,overcast clouds,No hotel found
12,12,Harper,LR,4.3750,-7.7169,80.19,84,89,6.04,overcast clouds,No hotel found
15,15,Namibe,AO,-15.1961,12.1522,72.54,71,99,4.83,overcast clouds,Hotel Chik Chik Namibe
17,17,Victoria,HK,22.2855,114.1577,77.25,68,0,4.00,clear sky,四季酒店 Four Seasons Hotel
23,23,Xai-Xai,MZ,-25.0519,33.6442,75.54,94,34,4.23,light rain,No hotel found
26,26,Mahebourg,MU,-20.4081,57.7000,79.09,89,75,10.36,light rain,Grand Bel Air
27,27,Caninde,BR,-4.3589,-39.3117,76.51,84,99,2.26,overcast clouds,Hotel Jardineira
30,30,Saint-Philippe,RE,-21.3585,55.7679,78.39,80,99,7.18,overcast clouds,Le Baril


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [36]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
0,0,Maiduguri,NG,11.8464,13.1603,77.05,25,41,7.87,scattered clouds,Nanne and Boi Suites
15,15,Namibe,AO,-15.1961,12.1522,72.54,71,99,4.83,overcast clouds,Hotel Chik Chik Namibe
17,17,Victoria,HK,22.2855,114.1577,77.25,68,0,4.00,clear sky,四季酒店 Four Seasons Hotel
26,26,Mahebourg,MU,-20.4081,57.7000,79.09,89,75,10.36,light rain,Grand Bel Air
27,27,Caninde,BR,-4.3589,-39.3117,76.51,84,99,2.26,overcast clouds,Hotel Jardineira


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [37]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [40]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Country", "Description"]
)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Description)